In [5]:
from openai import AzureOpenAI

deployment_name='gpt-4-0125-preview' 
client = AzureOpenAI(
    api_key="YUOR API KEY HERE",  
    api_version="2024-02-15-preview",
    base_url="YUOR BASE URL HERE"
    )

In [2]:
def judge(english, sentences):
    system_prompt = \
    f'''请根据准确性、流畅性、语法正确性和是否忠实于原文为以下英译中的结果排名
输出为每个翻译的名次,翻译的最好的排名为1，其次为2，翻译最差的排名为3
输出格式为：句子的标号+空格+排名+空格+排名理由+换行符
[示例]
1. 1 翻译准确，流畅，语法正确，忠于原文
2. 3 翻译不准确，不流畅，语法有较多错误
3. 2 翻译部分准确，流畅，语法有少量错误
'''

    unpack_sentences = "\n".join([f"{i+1}. {sentence}" for i, sentence in enumerate(sentences)])
    
    user_input = \
    f'''[英文原文]
{english} 
[汉语翻译]
{unpack_sentences}'''


    response = client.chat.completions.create(
        model=deployment_name,
        messages=[
            {"role": "system", "content": f"{system_prompt}"},
            {"role": "user", "content": f"{user_input}"},
        ]
    )

    return response.choices[0].message.content

In [3]:
s = judge("a quick brown fox jumps over the lazy dog", ["敏捷的棕色狐狸跨过懒狗", "狐狸跨过狗", "狗跨过了狐狸"])
print(s)

1. 1 翻译准确，流畅，语法正确，忠于原文
2. 2 翻译部分准确，但未能全面准确的描述原文中的信息，例如"快速的棕色狐狸"和"懒惰的狗"
3. 3 翻译不准确，更改了原文的意思


In [24]:
import re
def extract_scores(text, random_list):
    # 正则表达式来匹配每个评分的格式（数字+分）
    scores = re.findall(r'\d+\. (\d+)', text)
    res = [0] * len(random_list)
    for i, score in enumerate(scores):
        res[random_list[i] - 1] = int(score)
    return res

In [48]:
scores = []

import random
import pandas as pd

df = pd.read_excel("comparison_translated_text.xlsx")


for _, row in df.iterrows():
    eng, *sentences = row[1:]

    eng = eng.replace("\n", "")
    chis = [{'id': i, 'sentence': sentence.replace("\n", "")} for i, sentence in enumerate(sentences, 1)]
    random.shuffle(chis)

    results = judge(eng, [chi['sentence'] for chi in chis])
    print(results, [chi['id'] for chi in chis])

    score = extract_scores(results, [chi['id'] for chi in chis])

    print(score)
    scores.append(score)

    with open('grade.txt', 'a') as f:
        f.write(" ".join([str(s) for s in score]) + "\n")

1. 3 翻译部分准确，对一些专业词汇的处理较为生硬，不夏流畅，且有一些语法错误
2. 1 翻译准确，流畅，语法正确，忠于原文
3. 2 翻译大部分准确，流畅，语法基本正确，但一些专业词汇翻译不够准确 [2, 1, 3]
[1, 3, 2]
1. 3 翻译有误，表达不清，原文中的organohydrogels被错误翻译为有机水化物网状结构，而正确的翻译应为有机水凝胶。同时，对原文中的PAA/gelatin gels的翻译为PAA/羧甲基纤维素复配物也存在错误，正确的翻译应为PAA/明胶凝胶。
2. 1 翻译准确，流畅，语法正确，忠于原文，对于原始科学文章中的专业术语使用准确，阅读体验好。
3. 2 翻译基本准确，流畅，但在细节上存在瑕疵，如PAA/gelatin gels的翻译为PAA/明胶凝胶，没有翻译出网络的含义。 [2, 1, 3]
[1, 3, 2]
1. 1 翻译准确，流畅，语法正确，忠于原文
2. 3 翻译存在部分准确，但有些理解偏离原文，例如"梨果点"译为"小结构"，"stratum corneum"只能指角质层，不应译为“表皮层”，"cork tissue"只能译为“软木组织”不能译为“鳞状上皮组织”。此外，这段译文的流畅性和语法正确性较差。
3. 2 翻译大部分准确，流畅，语法正确，但对russeting的翻译“褐变”偏离原文，应译为“锈斑”。 [1, 2, 3]
[1, 3, 2]
1. 2 翻译准确，流畅，语法正确，但未忠实原文，原文中有无关SH卤化物，原文是关于polySH电解质的。
2. 3 翻译不准确，不忠实原文，原文中并未提及碘化钠和硫酸锂，语法正确但不流畅。
3. 1 翻译准确，流畅，语法正确，并且忠实于原文。 [3, 2, 1]
[1, 3, 2]
1. 3 翻译不准确，热耦合翻译成热场，涉及镧系元素离子相互作用和镧系元素-配体能量传递的一般物理学原理翻译成了涉及镧系元素离子相互作用和镧系元素配体能在热场中传递的一般物理原理
2. 1 翻译准确，流畅，语法正确，忠于原文
3. 2 翻译部分准确，涉及钨族元素阳离子交换作用和自由基化合物形成机理和原文的涉及镧系元素离子相互作用和镧系元素配体能在热场中传递的一般物理原理有出入 [1, 3, 2]
[3, 2, 1]
1. 2 翻译准确，流畅，语法正确无误，但未保持原文的信息完整，如原文中

In [49]:
import numpy as np
with open('grade.txt', 'a') as f:
    f.write("平均名次\n"+" ".join([str(s) for s in np.array(scores).sum(axis=0) / len(scores)]) + "\n")